In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import sqlite3 as sql
import re
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt


%matplotlib inline

In [ ]:
pd.set_option('display.max_rows', 100)
pd.options.display.max_colwidth = 3000

In [ ]:
# clean = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 0)
clean1 = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 1)
clean2 = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 2)
clean3 = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 3)
# clean4 = pd.read_excel('../data/Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_033122.xlsx')

In [ ]:
clean1.info()

In [ ]:
clean1 = clean1.dropna()
clean1.info()

In [ ]:
clean1.Precinct.value_counts()

In [ ]:
clean2.info()

In [ ]:
clean3.info()

In [ ]:
conn = sql.connect('../data/census.sqlite')
cur = conn.cursor()

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

In [ ]:
available_tables = (cur.fetchall())

In [ ]:
print(available_tables)

In [ ]:
cur.execute("SELECT * FROM B01001 LIMIT 5;")

In [ ]:
B01001 = (cur.fetchall())

In [ ]:
type(B01001)

In [ ]:
B01001

In [ ]:
pop_age_sex_df = pd.read_sql("SELECT * FROM B01001;", conn)

In [ ]:
pop_age_sex_df

In [ ]:
cur.execute("SELECT * FROM B03002 LIMIT 5;")
B03002 = (cur.fetchall())
pop_race_ethnicity_df = pd.read_sql("SELECT * FROM B03002;", conn)
pop_race_ethnicity_df

In [ ]:
cur.execute("SELECT * FROM B11001 LIMIT 5;")
B11001 = (cur.fetchall())
households_type_df = pd.read_sql("SELECT * FROM B11001;", conn)
households_type_df

In [ ]:
cur.execute("SELECT * FROM B11002 LIMIT 5;")
B11002 = (cur.fetchall())
pop_household_type_df = pd.read_sql("SELECT * FROM B11002;", conn)
pop_household_type_df

In [ ]:
cur.execute("SELECT * FROM B15002 LIMIT 5;")
B15002 = (cur.fetchall())
sex_educational_attainment_pop_25_up_df = pd.read_sql("SELECT * FROM B15002;", conn)
sex_educational_attainment_pop_25_up_df

In [ ]:
cur.execute("SELECT * FROM B19001 LIMIT 5;")
B19001 = (cur.fetchall())
household_income_df = pd.read_sql("SELECT * FROM B19001;", conn)
household_income_df

In [ ]:
cur.execute("SELECT * FROM B19013 LIMIT 5;")
B19013 = (cur.fetchall())
median_household_income_df = pd.read_sql("SELECT * FROM B19013;", conn)
median_household_income_df

In [ ]:
cur.execute("SELECT * FROM B19301 LIMIT 5;")
B19301 = (cur.fetchall())
per_capita_income_df = pd.read_sql("SELECT * FROM B19301;", conn)
per_capita_income_df

In [ ]:
cur.execute("SELECT * FROM B25024 LIMIT 5;")
B25024 = (cur.fetchall())
types_structure_df = pd.read_sql("SELECT * FROM B25024;", conn)
types_structure_df

In [ ]:
cur.execute("SELECT * FROM B25075 LIMIT 5;")
B25075 = (cur.fetchall())
value_owner_occupied_housing_units_df = pd.read_sql("SELECT * FROM B25075;", conn)
value_owner_occupied_housing_units_df

In [ ]:
cur.execute("SELECT * FROM S1701 LIMIT 5;")
S1701 = (cur.fetchall())
poverty_status_last_12_months_df = pd.read_sql("SELECT * FROM S1701;", conn)
poverty_status_last_12_months_df

In [ ]:
clean1